In [2]:
%matplotlib inline
import sys
import os
import io
import getpass
import psycopg2
import psycopg2.extras
import psycopg2.extensions
import numpy
import pandas
import PIL
import matplotlib
from matplotlib import pyplot

psycopg2.extensions.register_adapter(numpy.int64, psycopg2.extensions.AsIs)

In [3]:
# Connect to the database
# Don't hardcode username and password anywhere public, of course!
if os.path.isfile( "/global/homes/r/raknop/secrets/decatdb_decat_decat_ro" ):
    with open( "/global/homes/r/raknop/secrets/decatdb_decat_decat_ro" ) as ifp:
        dbuser, dbpasswd = ifp.readline().strip().split()
else:
    dbuser = 'decat_ro'
    dbpasswd = getpass.getpass( 'decat readonly database password: ' )
db = psycopg2.connect(dbname='decat', user=dbuser, password=dbpasswd, host='decatdb.lbl.gov',
                      cursor_factory=psycopg2.extras.RealDictCursor)

In [4]:
# Use a real text editor to get your candidate list with the quotes
# and commas...
candlist = [
'DC22iongd',
'DC22ionvl',
'DC22iopfn',
'DC22iopoi',
'DC22ioppt',
'DC22iopze',
'DC22ioqdo',
'DC22ioqol',
]

In [8]:
vtag = 1   # This is "latest"
rbtype = 2
q = ( "SELECT c.id,c.ra AS candra,c.dec AS canddec,od.ra,od.dec "
      "FROM candidates c "
      "INNER JOIN objects o ON c.id=o.candidate_id "
      "INNER JOIN objectdatas od ON o.id=od.object_id "
      "INNER JOIN objectdata_versiontag odvt ON od.id=odvt.objectdata_id AND odvt.versiontag_id=%(vtag)s "
      "INNER JOIN objectrbs r ON od.id=r.objectdata_id AND r.rbtype_id=%(rbtype)s "
      "WHERE c.id IN %(candlist)s" )
db.rollback()
with db.cursor() as cursor:
    cursor.execute( q, { "vtag": vtag, "rbtype": rbtype, "candlist": tuple(candlist) } )
    df = pandas.DataFrame( cursor.fetchall() )
df


,id,candra,canddec,ra,dec
0,DC22iongd,34.197845,-4.418514,34.197855,-4.418473
1,DC22iongd,34.197845,-4.418514,34.197837,-4.418501
2,DC22iongd,34.197845,-4.418514,34.197858,-4.418542
3,DC22iongd,34.197845,-4.418514,34.197865,-4.418495
4,DC22iongd,34.197845,-4.418514,34.197851,-4.418507
...,...,...,...,...,...
97,DC22ioqol,36.320294,-4.207819,36.320313,-4.207824
98,DC22ioqol,36.320294,-4.207819,36.320306,-4.207862
99,DC22ioqol,36.320294,-4.207819,36.320299,-4.207850
100,DC22ioqol,36.320294,-4.207819,36.320317,-4.207865


In [9]:
# Take a mean of all the high r/b detections of the candidates'
# positions; those positions should be better positions than
# candra and canddec, which is just from the first detection.
dfmean = df.groupby( 'id' ).apply( 'mean' )
dfmean

,candra,canddec,ra,dec
id,,,,
DC22iongd,34.197845,-4.418514,34.197856,-4.418506
DC22ionvl,34.072297,-4.177322,34.072311,-4.177322
DC22iopfn,35.844771,-6.537498,35.844774,-6.537512
DC22iopoi,35.840480,-5.723967,35.840463,-5.723942
DC22ioppt,36.136358,-6.645094,36.136351,-6.645112
DC22iopze,37.272530,-4.708446,37.272440,-4.708498
DC22ioqdo,37.101106,-4.860344,37.101129,-4.860376
DC22ioqol,36.320294,-4.207819,36.320305,-4.207838


In [11]:
# Look at the offsets between the mean positions and the candra/canddec in arceseconds
dfmean['Δra'] = ( dfmean.ra - dfmean.candra ) * 3600. * numpy.cos( dfmean.dec * numpy.pi / 180. )
dfmean['Δdec'] = ( dfmean.dec - dfmean.canddec ) * 3600.
dfmean

,candra,canddec,ra,dec,Δra,Δdec
id,,,,,,
DC22iongd,34.197845,-4.418514,34.197856,-4.418506,0.038456,0.028485
DC22ionvl,34.072297,-4.177322,34.072311,-4.177322,0.050318,-0.001586
DC22iopfn,35.844771,-6.537498,35.844774,-6.537512,0.010120,-0.052823
DC22iopoi,35.840480,-5.723967,35.840463,-5.723942,-0.061833,0.090230
DC22ioppt,36.136358,-6.645094,36.136351,-6.645112,-0.027667,-0.066285
DC22iopze,37.272530,-4.708446,37.272440,-4.708498,-0.323023,-0.186210
DC22ioqdo,37.101106,-4.860344,37.101129,-4.860376,0.081416,-0.113206
DC22ioqol,36.320294,-4.207819,36.320305,-4.207838,0.037892,-0.068871
